In [1]:
import plotly.express as px
import pandas as pd

### ISO-3 country codes

In [2]:
iso_codes = pd.read_csv('https://gist.githubusercontent.com/tadast/8827699/raw/f5cac3d42d16b78348610fc4ec301e9234f82821/countries_codes_and_coordinates.csv')
iso_codes = iso_codes.set_index('Country')
iso_codes = iso_codes['Alpha-3 code']
iso_codes = iso_codes.replace(regex={'\"': '', ' ': ''})

iso_codes['Iran'] = 'IRN'
iso_codes['US'] = 'USA'
iso_codes['Syria'] = 'SYR'
iso_codes['Congo (Kinshasa)'] = 'COD'
iso_codes['Korea, South'] = 'KOR'
iso_codes['Taiwan*'] = 'TWN'
iso_codes['Congo (Brazzaville)'] = 'COG'
iso_codes['Laos'] = 'LAO'
iso_codes["Cote d'Ivoire"] = 'CIV'
iso_codes['Czechia'] = 'CZE'
iso_codes['Tanzania'] = 'TZA'

### Get the data from url

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

data = data.drop(columns=['Province/State','Lat', 'Long'])
data = data.groupby('Country/Region').agg('sum')

date_list = list(data.columns[7:])

data['country'] = data.index
data['iso_code'] = data['country'].apply(lambda x: iso_codes.loc[x] if x in iso_codes.index else None)

### The countries that don't have iso code

In [4]:
data.loc[data["iso_code"].isnull()]

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,...,5/20/21,5/21/21,5/22/21,5/23/21,5/24/21,5/25/21,5/26/21,5/27/21,5/28/21,5/29/21,5/30/21,5/31/21,6/1/21,6/2/21,6/3/21,6/4/21,6/5/21,6/6/21,6/7/21,6/8/21,6/9/21,6/10/21,6/11/21,6/12/21,6/13/21,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21,6/21/21,6/22/21,6/23/21,6/24/21,6/25/21,6/26/21,country,iso_code
Country/Region,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Cabo Verde,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,28898,29033,29198,29334,29411,29549,29763,29939,30122,30273,30359,30439,30523,30694,30774,30907,31003,31097,31147,31225,31339,31433,31512,31571,31571,31647,31704,31783,31858,31910,31964,32002,32039,32111,32192,32257,32313,32344,Cabo Verde,None
Diamond Princess,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,61,61,64,135,135,175,175,218,285,355,454,542,621,634,634,634,691,691,691,705,705,705,705,705,...,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,Diamond Princess,None
Eswatini,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,18539,18546,18550,18551,18557,18570,18572,18582,18587,18589,18591,18595,18601,18618,18623,18632,18636,18653,18664,18680,18694,18705,18723,18732,18736,18746,18754,18768,18784,18800,18811,18813,18831,18854,18898,18947,18989,19018,Eswatini,None
Holy See,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,Holy See,None
Kosovo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,106712,106770,106770,106770,106770,106861,106905,106928,106946,106946,106997,107002,107021,107039,107051,107058,107058,107058,107099,107099,107125,107125,107125,107125,107170,107175,107175,107192,107192,107192,107192,107192,107247,107247,107247,107299,107299,107299,Kosovo,None
MS Zaandam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,MS Zaandam,None
Micronesia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,Micronesia,None
Moldova,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,254379,254499,254601,254676,254741,254814,254908,254985,255060,255105,255145,255186,255241,255285,255354,255402,255432,255453,255501,255541,255601,255661,255715,255758,255797,255830,255878,255937,255994,256054,256103,256140,256187,256232,256324,256387,256475,256527,Moldova,None
North Macedonia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,...,154937,154982,155028,155063,155067,155071,155117,155169,155217,155246,155269,155272,155304,155345,155364,155379,155407,155417,155418,155441,155469,155487,155507,155528,155536,155540,155568,155583,155593,155612,155616,155624,155627,155640,155645,155655,155663,155669,North Macedonia,None


### Total case count -> daily case count

In [5]:
data[data.columns[:-2]] = data[data.columns[:-2]] - data[data.columns[:-2]].shift(1, axis=1, fill_value=0)

### 7 days moving average

In [6]:
data[data.columns[:-2]] = data[data.columns[:-2]].rolling(window=7, axis=1).mean()
data = data.drop(columns=data.columns[:7])
data[data.columns[:-2]] = data[data.columns[:-2]].astype('int')

### Last form of data frame

In [7]:
df_long = pd.melt(data, id_vars=['country', 'iso_code'], value_vars=date_list)
df_long = df_long.rename(columns={"variable": "date", "value": "case_count"})

### Plot the data (the colors based on cases)

In [8]:
fig = px.choropleth(df_long,                          
                     locations="iso_code", 
                     color="case_count",
                     hover_name="country",
                     animation_frame="date",
                     projection="natural earth",
                     color_continuous_scale = "Peach",
                     range_color=[0, 5000]
                     )

fig.show()
# fig.write_html("case_map.html")   If you want to write the map as a html page

Output hidden; open in https://colab.research.google.com to view.